# Classifying Microscopic Histopathology Images with PyTorch

- [View Solution Notebook](./solutions.html)
- [View Project Page](https://www.codecademy.com/content-items/b68fc937824d450e8bce11e24126e1e8)

**Setup - Libraries and Custom PCam Dataset Loading Class**

Run the cell below to import the libraries and the custom PCam dataset loading class.

In [2]:
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
torch.manual_seed(42)


class PCamDataset(Dataset):
    """
    Custom Dataset for loading the microscopic histopathology images within the PCam dataset
    """
    def __init__(self, csv_file, transform=None, num_samples=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations
            transform (callable, optional): Optional transform to be applied on a sample
            num_samples (int, optional): Number of samples to load. If None, loads all samples
        """
        self.annotations = pd.read_csv(csv_file)
        if num_samples is not None:
            self.annotations = self.annotations.head(num_samples)
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        # Get image path and label
        img_path = self.annotations.iloc[idx, 0]
        label = self.annotations.iloc[idx, 1]
        
        # Load and convert image
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
            
        if self.transform:
            image = self.transform(image)
            
        # Convert label to float
        label = torch.tensor(label, dtype=torch.float)
            
        return image, label

## Task Group 1 - CNN Pre-Processing

Let's first train a CNN model!

### Task 1

Let's use `transforms.Compose([])` to create the image pre-processing pipeline to apply the following transformations **and** augmentations to the training set:
- resizes images to `96`x`96` pixels
- randomly flips images horizontally
- randomly rotates images `15` degrees clockwise or counterclockwise
- adjusts brightness and contrast within 20% (or `0.20`)
- converts the image datatype to a PyTorch tensor with values ranging from `[0.0, 1.0]`
- normalize the pixel values within the **3** color channels to have a mean of `0.5` and a standard deviation of `0.5`

Save the training set pre-processing pipeline to the variable `train_transform`.

**Note**: be sure to apply the transformations in the correct order!

In [3]:
import torchvision.transforms as transforms
torch.manual_seed(42)

train_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

### Task 2

Next, load the PCam **training set** while applying the training transformation pipeline to each image using the custom `PCamDataset` class with the following parameters:
- `csv_file` to specify the training set located in the path `'data/train_labels.csv'`
- `transform` to apply the `train_transform` pre-processing pipeline

Save the PCam training set to the variable `train_dataset`.

In [4]:
torch.manual_seed(42)

train_dataset = PCamDataset(
    csv_file='data/train_labels.csv',
    transform=train_transform
)

# first training image
image, label = train_dataset[0]
print("First image:", image)
print("First label:", label)
print("Training size:", len(train_dataset))

First image: tensor([[[-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         ...,
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137]],

        [[-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         ...,
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137]],

        [[-0.9137, -0.9137, -0.9137,  ..., -0.9137, -0.9137, -0.9137],
         [-0.9137, -0.9137, -0.9

### Task 3

Create an iterable using the PyTorch `DataLoader` utility class that allows us to efficiently load the training set images in batches during training:

- set to load `8` training images per batch
- be sure to shuffle the training images

Save the dataloader iterable to the variable `train_dataloader`.

In [5]:
from torch.utils.data import DataLoader
torch.manual_seed(42)

train_dataloader = DataLoader(
    train_dataset, 
    batch_size=8, 
    shuffle=True
)

# show output - DO NOT MODIFY
first_batch = next(iter(train_dataloader))
images, labels = first_batch
print("Batch shape:", images.shape)
print("Training labels:", labels[:10])

Batch shape: torch.Size([8, 3, 96, 96])
Training labels: tensor([1., 1., 0., 1., 0., 1., 1., 0.])


### Optional Task

Visualize the images in the first training batch:

In [ ]:
# Visualize the first training batch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def imshow(img):
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get a batch of training data
dataiter = iter(train_dataloader)
images, labels = next(dataiter)
batch_size = 8

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(' '.join('%5s' % labels[j].item() for j in range(batch_size)))

### Task 4

Next, let's load and pre-process the PCam **validation** and **testing** set.

Remember, we don't need to apply the augmentation techniques used to pre-process the training set. But we do need to apply the same transformations (resizing, normalization, and tensor conversion) to ensure the testing set images are consistent with the training set images and are suitable for use as model inputs.

**A.** Create the image pre-processing pipeline using the `transforms.Compose([])` class from the `torchvision` library that applies the following transformations to the testing set:
- resizes images to `96`x`96` pixels
- converts the image datatype to a PyTorch tensor with values ranging from `[0.0, 1.0]`
- normalize the pixel values within the 3 color channels to have a mean of `0.5` and a standard deviation of `0.5`

Save the validation/testing pre-processing pipeline to the variable `val_test_transform`.

**Note**: be sure to apply the transformations in the correct order!

In [ ]:
val_test_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

### Task 5

**A.** Next, load the PCam **validation set** while applying the validation/testing pre-processing pipeline:
- `csv_file` to specify the validation set located in the path `'data/validation_labels.csv'`
- `transform` to apply the `val_test_transform` pre-processing pipeline

Save the loaded validation set to the variable `val_dataset`.

**B.** Lastly, create an iterable using the PyTorch `DataLoader` utility class that allows us to efficiently load the validation set images in batches during evaluation:

- set to load `32` images per batch
- be sure **not** to shuffle the images

Save the dataloader iterable to the variable `val_dataloader`.

In [ ]:
val_dataset = PCamDataset(
    csv_file='data/validation_labels.csv',
    transform=val_test_transform
)

val_dataloader = DataLoader(
    val_dataset, 
    batch_size=32, 
    shuffle=False
)
print("Validation size:", len(val_dataset))

### Task 6

**A.** Next, load the PCam **testing set** while applying the validation/testing pre-processing pipeline:
- `csv_file` to specify the validation set located in the path `'data/test_labels.csv'`
- `transform` to apply the `val_test_transform` pre-processing pipeline

Save the loaded validation set to the variable `test_dataset`.

**B.** Lastly, create an iterable using the PyTorch `DataLoader` utility class that allows us to efficiently load the testing set images in batches during evaluation:

- set to load `32` images per batch
- be sure **not** to shuffle the images

Save the dataloader iterable to the variable `test_dataloader`.

In [ ]:
test_dataset = PCamDataset(
    csv_file='data/test_labels.csv',
    transform=val_test_transform
)

test_dataloader = DataLoader(
    test_dataset, 
    batch_size=32, 
    shuffle=False
)
print("Testing size:", len(test_dataset))

## Task Group 2 - CNN Training and Evaluation

### Task 7

Let's now a create a CNN architecture as a class named `SimpleCNN` using the `nn.Module`.

**A.** Define the `__init__` method with the following CNN layers:
1. `self.conv1` is the first convolutional layer with `3` input channels, `32` output channels, `3x3` filter sizes, and `1` padding
2. `self.conv2` is the second convolutional layer with `32` input channels, `64` output channels, `3x3` filter sizes, and `1` padding
3. `self.conv3` is the third convolutional layer with `64` input channels, `128` output channels, `3x3` filter sizes, and `1` padding 
4. `self.fc1` is the first fully connected layer with `18432` input nodes and `256` output nodes
    - `18432` corresponds to the length of the flattened 1-D vector after last max pooling layer (`128 x 12 x 12 = 18432`)
5. `self.fc2` is the second fully connected layer with `256` input nodes and `1` output node

**B.** Define the `forward` method that processes each image `x` with the forward operations in following order:

1. Pass the image through the first convolutional layer and then apply the ReLU activation function
2. Pass the first activated convoluted output to a max pooling layer with a 2x2 filter
3. Pass the first max pooling output to the second convolutional layer and then apply the ReLU activation function
4. Pass the second activated convoluted output to a 2nd max pooling layer with a 2x2 filter
5. Pass the second max pooling output to the third convolutional layer and then apply the ReLU activation function
6. Pass the third activated convoluted output to a 3rd max pooling layer with a 2x2 filter
7. Flatten the third max pooling output into a tensor (with batch size)
8. Pass the flattened tensor to the first fully connected layer and then apply the ReLU activation function
9. Pass the activated output to the second fully connected layer through a Sigmoid activation function
    - **Hint:** Use `x = torch.sigmoid(self.fc2(x)).squeeze(1)`
10. Return Sigmoid activated output

**C.** Create an instance of the CNN model class and save it to the variable `cnn_model`.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 12 * 12, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x)).squeeze(1)
        return x
        
# Instantiate the model
cnn_model = SimpleCNN()

### Task 8

Set the CNN hardware device to GPU/CPU.

**A.** Create the `device` variable that detects whether GPU (`'cuda'`) or CPU is available.
- use `torch.device()` to detect the device
- use `torch.cuda.is_available()` to check if GPU is available
    - if GPU is available, return the string `'cuda'`
    - if not, return the string `'cpu'`

**B.** Move the `cnn_model` to the available device.

In [ ]:
# Set device to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_model.to(device)
print(device)

### Task 9

Let's initialize the loss function and optimizer for training using the `torch.optim` module.

**A.** Create an instance of the binary-entropy loss function `nn.BCELoss()` in PyTorch and save it to the variable `criterion`.

**B.** Create an instance of the Adam optimizer in PyTorch with a learning rate of `0.0005` and save it to the variable `optimizer`.
- be sure to optimize the parameters in the CNN model we instantiated

In [ ]:
import torch.optim as optim
torch.manual_seed(42)

criterion = nn.BCELoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.0005)

# show output
print("Loss function:", criterion)
print("Optimizer:", optimizer)

### Task 10

Now, let's create a training loop that:
- trains the CNN model on training set and keeps track of the training loss
- keeps track of the validation loss on the training set

**A.** Initialize the following empty lists to keep track the training and validation losses per epoch:
- `train_losses` to keep track of the training loss
- `val_losses` to keep track of the validation loss

**B.** Train the CNN on `5` epochs by assigning the value `5` to the variable `num_epochs`.

**C.** Initialize the following variables to `0` to keep track of during training:
- `total_train_loss` to keep track of the total training loss
- `total_val_loss` to keep track of the total validation loss

**D.** Build the training section:
- loop through the training batch images and labels
- within each training batch:
    - place the images to the GPU device 
    - reset the gradients to zero
    - pass the batch through the CNN forward pass
    - calculate the training loss
    - backpropagate the loss
    - adjust the weights and biases
    - update the total training loss

**E.** Within the validation section, evaluate the validation set performance:
- loop through the validation batch images and labels
- within each validation batch:
    -  place the images to the GPU device
    -  pass the batch through the CNN forward pass
    -  calculate the validation loss
    -  update the total validation loss

In [ ]:
torch.manual_seed(42)

train_losses = []
val_losses = []

num_epochs = 5
for epoch in range(num_epochs):
    total_train_loss = 0
    total_val_loss = 0

    # Training
    cnn_model.train()
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        
    # Validation
    cnn_model.eval()
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = cnn_model(images)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

    # DO NOT MODIFY
    # Calculate average losses
    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_val_loss = total_val_loss / len(val_dataloader)

    # Save the average training and validation losses
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    
    # Print training and validation losses
    print(f"Epoch [{epoch+1}/{num_epochs}]: Training Loss: {avg_train_loss:.4f} - Validation Loss: {avg_val_loss:.4f}")

<details><summary style="display:list-item; font-size:16px; color:blue;">Hint: Tracking the training and validation losses</summary>
    
You can add this code snippet to the training loop to track the training and validation losses:

```py
    # Calculate average losses
    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_val_loss = total_val_loss / len(val_dataloader)

    # Save the average training and validation losses
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    
    # Print training and validation losses
    print(f"Epoch [{epoch+1}/{num_epochs}]: Training Loss: {avg_train_loss:.4f} - Validation Loss: {avg_val_loss:.4f}")
```

### Optional Task

Visualize the training and validation losses during training.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_losses, label='Training Loss', marker='o')
plt.plot(epochs, val_losses, label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

### Task 11

Next, let's use our trained CNN that we've loaded and saved to the variable `cnn_model` to generate predictions (labels and probabilities) on the testing set images.

**Note:** Since we're using the binary cross-entropy loss, the outputs from the forward pass are already converted to probabilities (thanks to PyTorch).

**A.** Create empty lists for the variables `test_pred_probs` and `test_pred_labels` to save the predicted probabilities and labels.

**B.** Within the `torch.no_grad()` statement:
- loop through the testing images and labels in the testing dataloader iterable
- pass the images through the forward pass of the CNN to generate outputs (probabilities)
- add the probabilities to the list `test_pred_probs`
    - **Hint:** Use `test_pred_probs.extend(outputs.cpu().numpy())`
- use the `torch.round()` function to round the probabilities to their predicted labels
    - **Hint:** probabilities `>.50` are assigned label `1`, else they are assigned label `0`
- add the predicted labels to the list `test_pred_labels`
    - - **Hint:** Use `test_pred_labels.extend(pred_labels.cpu().numpy())`
     
**C.** Convert `test_pred_probs` and `test_pred_labels` to NumPy arrays using `np.array`

In [ ]:
import numpy as np

test_pred_probs = []
test_pred_labels = []

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        # Get model outputs (already has sigmoid activation)
        outputs = cnn_model(images)
        
        # Store probabilities
        test_pred_probs.extend(outputs.cpu().numpy())
        
        # Get predicted labels (threshold at 0.5)
        pred_labels = torch.round(outputs)
        test_pred_labels.extend(pred_labels.cpu().numpy())

# Convert to numpy arrays for easier handling
test_pred_probs = np.array(test_pred_probs)
test_pred_labels = np.array(test_pred_labels)

# Show output
print("First image predicted probability:", test_pred_probs[0])
print("First image predicted label:", test_pred_labels[0])

### Task 12 

Let's evaluated the trained CNN by generating a classification report.

**A.** Obtain the testing set labels within the testing set dataloader.
- initialize an empty list `test_true_labels = []` to store the true labels
- loop through the images and labels in `test_dataloader` and use `.extend()` to add each label (as a NumPy array to the `test_true_labels` list

**B.** Create a list named `pcam_classes` that stores the PCam class name
- index `0` should reference the `'Normal'` class
- index `1` should reference the `'Tumor'` class

**C.** Generate a classification report using the predicted and true labels. Save the report to the variable `report`.
- be sure to format the report by passing the PCam classes into the parameter `target_names=`

Print the classification report.

In [ ]:
from sklearn.metrics import classification_report

test_true_labels = []
for _, labels in test_dataloader:
    test_true_labels.extend(labels.numpy())

# Define class names for PCam
pcam_classes = ['Normal', 'Tumor']

# Generate classification report
report = classification_report(y_true=test_true_labels, 
                               y_pred=test_pred_labels,
                               target_names=pcam_classes)

# show report
print(report)

**Conclusion**

Nice! Congrats on finishing the project on building a CNN to detect tumors within images of metastatic tissue in the PatchCamelyon (PCam) dataset! There's definitely a lot of room for improvement and we encourage you to use your skills to explore different techniques to enhance the vision models.

Here are some areas for improvement:
try experimenting with different augmetation techniques (it may be useful to research techniques that are specific to medical imaging)
- explore different model architectures in the CNN (adding additional layers or changing the parameters of the convolution layers)
- test different loss functions or optimizers with varying learning rates

While we've built a model for detecting tumors, this project primarily serves as an educational foundation for understanding how deep learning models can be applied to medical images. 

In the real-world, medical AI systems  require extensive testing before applying them to real patients. For example, here are just a few crucial things to consider:

- optimizing for accuracy may not always be appropriate
- understanding the trade-offs between precision and recall (false positives and false negatives)
- robustness across different imaging conditions
- the importance of model interpretability for clinical trust among practitioners and patients
- the importance of consulting with medical professionals to provide valuable insights with their expertise
er memory.

Happy coding!ing!

In [ ]:
!jupyter nbconvert --to html solutions.ipynb